In [1]:
import plotly.express as px
from pyspark.sql.functions import *

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('flights').getOrCreate()

23/01/05 11:51:30 WARN Utils: Your hostname, MacBook-Air-di-Teodoro.local resolves to a loopback address: 127.0.0.1; using 192.168.240.184 instead (on interface en0)
23/01/05 11:51:30 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/01/05 11:51:31 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/01/05 11:51:31 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
# read the schema from the json file
import json


with open("../../util/schema.json","r") as f:
    schema = StructType.fromJson(json.load(f))

In [4]:

df_cleaned = spark.read.csv('../../data.nosync/cleaned/cleaned_flights.csv/',schema=schema,header=True)

In [5]:
df = df_cleaned

In [8]:
# add a new column delay that is 1 if ArrDelay>0 and 0 otherwise
df = df.withColumn('delay',when(df['ArrDelay']>0,1).otherwise(0))

In [12]:
df_agg = df.groupBy('FlightDate').\
        agg({
            '*' : 'count',
            'delay' : 'count',
            'ArrDelay' : 'sum'
        })

# rename the columns
df_agg = df_agg.withColumnRenamed('count(1)','count')
df_agg = df_agg.withColumnRenamed('count(delay)','delay_count')
df_agg = df_agg.withColumnRenamed('sum(ArrDelay)','delay_sum')


In [15]:
df_agg.columns

['FlightDate', 'delay_count', 'count', 'delay_sum']

In [29]:
# convert to pandas, save to csv
df_agg.toPandas().to_csv('../../util/textual_queries.csv',index=False)

/opt/homebrew/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:248: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)


In [25]:
import pandas as pd
textual = pd.read_csv("../../util/textual_queries.csv")

In [26]:
# convert textual["FlightDate"] to timestamp
textual["FlightDate"] = pd.to_datetime(textual["FlightDate"])

In [27]:
# save to pickle
textual.to_pickle("../../util/textual_queries.pkl")

In [28]:
# filter textual by dates
textual.columns

Index(['FlightDate', 'delay_count', 'count', 'delay_sum'], dtype='object')